# Model Predictive Control (MPC) - Basics
### Notebook 1 of 3: Fundamentals

---
This notebook introduces **Model Predictive Control (MPC)** step by step, starting from **Linear Quadratic Regulation (LQR)**.

We will cover:
- Motivation: Why MPC?
- PID vs LQR vs MPC
- LQR derivation (theory + Riccati solution)
- Example: Double integrator system under LQR
- Transition from LQR to MPC
- Brunton-style diagrams (prediction horizon, block diagrams)
- Exercises for tuning $Q$ and $R$


## 1. Motivation

- **PID**: reactive, hand-tuned, no explicit model.
- **LQR**: optimal control for linear systems, fixed feedback gain.
- **MPC**: uses the model *online*, optimizes at each step, handles constraints.

### Key Question
Why do we need MPC when LQR already gives an optimal solution?
- Because LQR cannot enforce constraints.
- MPC can predict the future and act conservatively if constraints might be violated.


## 2. LQR Theory

We consider the linear system:

$$
x_{k+1} = A x_k + B u_k.
$$

We minimize the quadratic cost:

$$
J = \sum_{k=0}^\infty (x_k^T Q x_k + u_k^T R u_k).
$$

Solution: feedback law

$$
u_k = -Kx_k, \quad K = (R + B^T P B)^{-1} B^T P A.
$$

where $P$ solves the **Discrete Algebraic Riccati Equation (DARE)**:

$$
P = A^T P A - A^T P B (R + B^T P B)^{-1} B^T P A + Q.
$$


In [1]:
import numpy as np
import scipy.linalg as la

# Double integrator system
A = np.array([[1, 1],
              [0, 1]])
B = np.array([[0],
              [1]])
Q = np.eye(2)
R = np.array([[1]])

# Solve Discrete Algebraic Riccati Equation
P = la.solve_discrete_are(A, B, Q, R)
K = la.inv(B.T @ P @ B + R) @ (B.T @ P @ A)
print("LQR Gain K =", K)


LQR Gain K = [[0.42208244 1.24392885]]


### Exercise 1
- Try changing $Q$ and $R$.
- Example: $Q=10I$, $R=1$ vs. $Q=I$, $R=10$.
- Which case produces more aggressive control?

### Exercise 1 - Solution
Case 1: $Q = 10I, \; R = 1$  
- Large $Q$: strong penalty on state error.  
- Small $R$: weak penalty on control effort.  
- $\Rightarrow$ **Aggressive control**: large inputs, fast convergence.  

Case 2: $Q = I, \; R = 10$  
- Small $Q$: mild penalty on state error.  
- Large $R$: strong penalty on control effort.  
- $\Rightarrow$ **Conservative control**: smoother inputs, slower convergence.  

**Conclusion:**  
- Large $Q$, small $R$ $\;\Rightarrow\;$ Aggressive control  
- Small $Q$, large $R$ $\;\Rightarrow\;$ Conservative control  



## 3. LQR Simulation (Double Integrator)

We simulate the closed-loop system with $u = -Kx$.


In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def lqr_demo(Q_pos=1.0, Q_vel=1.0, R_val=1.0):
    Q = np.diag([Q_pos, Q_vel])
    R = np.array([[R_val]])
    P = la.solve_discrete_are(A, B, Q, R)
    K = la.inv(B.T @ P @ B + R) @ (B.T @ P @ A)
    
    x = np.array([[5],[0]])
    steps = 30
    x_hist = []
    for t in range(steps):
        u = -K @ x
        x = A @ x + B @ u
        x_hist.append(x.flatten())
    x_hist = np.array(x_hist)
    
    plt.figure(figsize=(6,3))
    plt.plot(x_hist[:,0], label="Position")
    plt.plot(x_hist[:,1], label="Velocity")
    plt.title(f"LQR with Q=[{Q_pos}, {Q_vel}], R={R_val}")
    plt.legend()
    plt.show()

interact(lqr_demo, Q_pos=(0.1, 20, 0.5), Q_vel=(0.1, 20, 0.5), R_val=(0.1, 20, 0.5));


interactive(children=(FloatSlider(value=1.0, description='Q_pos', max=20.0, min=0.1, step=0.5), FloatSlider(va…

## 4. From LQR to MPC

- **LQR**: unconstrained, infinite horizon, fixed $K$.
- **MPC**: constrained, finite horizon, solved repeatedly.

### MPC Cost Function
$$
J = \sum_{i=0}^{N-1} (x_{k+i}^T Q x_{k+i} + u_{k+i}^T R u_{k+i}) + x_{k+N}^T P x_{k+N}.
$$

Constraints:
- $u_{min} \le u \le u_{max}$
- $x_{min} \le x \le x_{max}$


In [ ]:
# Diagram: Prediction Horizon
N = 5
x_vals = np.arange(N+1)
plt.figure(figsize=(8,3))
plt.plot(x_vals, np.zeros_like(x_vals), 'ko-', label='Predicted trajectory')
for i in range(N):
    plt.arrow(x_vals[i], 0, 1, 0, head_width=0.05, head_length=0.2, fc='blue', ec='blue')
plt.axvline(x=0, color='red', linestyle='--', label='Current state')
plt.axvline(x=N, color='green', linestyle='--', label='End of horizon')
plt.title('Prediction Horizon in MPC')
plt.xlabel('Future time steps')
plt.yticks([])
plt.legend()
plt.show()


## 5. Block Diagram Comparison

We compare LQR vs MPC architectures.


In [ ]:
import matplotlib.patches as mpatches

fig, axes = plt.subplots(1,2, figsize=(12,4))

# LQR block diagram
axes[0].set_title('LQR Architecture')
axes[0].arrow(0.1,0.5,0.3,0,head_width=0.05, head_length=0.05, fc='k')
axes[0].text(0.05,0.5,'State x')
rect = mpatches.Rectangle((0.4,0.4),0.2,0.2,fill=False)
axes[0].add_patch(rect)
axes[0].text(0.5,0.5,'-K')
axes[0].arrow(0.6,0.5,0.3,0,head_width=0.05, head_length=0.05, fc='k')
axes[0].text(0.95,0.5,'u')
axes[0].axis('off')

# MPC block diagram
axes[1].set_title('MPC Architecture')
axes[1].arrow(0.05,0.5,0.25,0,head_width=0.05, head_length=0.05, fc='k')
axes[1].text(0.0,0.5,'State x')
rect2 = mpatches.Rectangle((0.3,0.4),0.4,0.2,fill=False)
axes[1].add_patch(rect2)
axes[1].text(0.5,0.5,'Optimizer\n(min cost)')
axes[1].arrow(0.7,0.5,0.25,0,head_width=0.05, head_length=0.05, fc='k')
axes[1].text(0.95,0.5,'u')
axes[1].axis('off')

plt.show()


### Exercise 2
- Explain in your own words: why is MPC more flexible than LQR?
- Hint: Think about constraints and re-solving.

### Exercise 2 - Solution
- **LQR**: one-time optimization → fixed feedback gain, no constraints.  
- **MPC**: re-solves optimization every step, adapts to states, and enforces constraints.  

$\Rightarrow$ MPC is more flexible than LQR.  
